# WIP: Video generation

In [3]:
# General

import numpy
import pandas

# Audio

from sunflower.sunflower.song_loader import Song, load_from_disk
from sunflower.sunflower.song_analyzer import SongAnalyzer
from sunflower.sunflower.utils import export_wav
from sunflower.sunflower.song_visualizer import visualize_waveform
import plotly.graph_objects as go
import numpy as np
import librosa
import soundfile as sf

# Video

from moviepy.editor import *
from moviepy.audio.AudioClip import AudioArrayClip
import moviepy
import pygame

# Autoreloading for easier development

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Audio

In [4]:
# Loading example file

raw_audio, extension = load_from_disk("assets/laylow_casting_slowed.mp3")

song = Song(raw_audio, extension)

song.print_attributes()

waveform: [[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]], mono_waveform: [0. 0. 0. ... 0. 0. 0.], extension: mp3, channels: 2, sr: 44100, sample_width: 2


In [5]:
# Analyzing song

song_analyzer = SongAnalyzer(song)
song_analyzer.detect_tempo()

print(song_analyzer.tempo)

224.69429347826087


## Video

In [6]:
def squared_size(clip): 
    
    # Resize
    clip=clip.crop(x_center=int(clip.w/2), 
                   y_center=int(clip.h/2),
                   width=min(clip.w,clip.h), 
                   height=min(clip.w,clip.h))
    
    return(clip)

def invert_green_blue(image):
    return image[:,:,[0,1,1]]



In [7]:
audioclip = AudioArrayClip(song.waveform , 
                           fps=song.sr)

In [8]:
# Settings

WIDTH = 600
HEIGHT = 600

screensize = (WIDTH,HEIGHT)
prop_center = 0.8

BPM = 112
duration = 60/BPM

artist_name = "Laylow"
track_name = "Casting"

# Fonts

textwidth = 0.2*WIDTH
textheight = 0.2*HEIGHT

In [31]:
txtClip_artist = TextClip(artist_name,color='white', font="Helvetica",
                   kerning = 5, size=(textwidth,None))

txtClip_track = TextClip(track_name,color='LightPink', font="Helvetica-bold",
                   kerning = 5, size=(textwidth,None))

# Load all the videos

# Zankyou

clip_og = VideoFileClip("assets/zankyou.mp4")
clip_og=squared_size(clip_og)
clip_og=clip_og.resize(screensize).resize(0.60)

# VHS

clip_vhs = VideoFileClip("assets/vhs.mp4")
clip_vhs=squared_size(clip_vhs)
clip_vhs=clip_vhs.resize(screensize)

masked_clip = clip_vhs.subclip(500,550).fx(moviepy.editor.vfx.mask_color, color=[0, 0, 0], thr=100, s=5)

# Background

clip_background = VideoFileClip("assets/vhs.mp4")
clip_background =squared_size(clip_background)
clip_background=clip_background.resize(screensize)

# Son

clip = clip_vhs.subclip(120,120+4*duration)

cuts = [51,53]
cuts = [7,13,30,39,47,51,58,64,83]

clip=concatenate_videoclips([clip,clip_og.subclip(cuts[0],cuts[1]+duration)])
                             
for i in range(1,len(cuts)-1):
    

    clip=concatenate_videoclips([clip,clip_og.subclip(cuts[i],cuts[i+1]+duration)])
    

# Looping the clip
clip = clip.loop(n=2)

# Say that you want it to appear 10s at the center of the screen
txtClip_artist = txtClip_artist.set_pos((WIDTH/2 - txtClip_artist.w/2,0.1*HEIGHT)).set_start(duration*2).set_duration(duration)

# Say that you want it to appear 10s at the center of the screen
txtClip_track = txtClip_track.set_pos((WIDTH/2 - txtClip_track.w/2,0.25*HEIGHT)).set_start(duration*2).set_duration(duration)


# Overlay the text clip on the first video clip
video = CompositeVideoClip([clip.set_position("center"), txtClip_artist, txtClip_track,masked_clip])

video=video.set_audio(audioclip)



# Cut de la video 

video = video.subclip(0,50)


In [34]:
video.preview(fps=24)

Interrupt


[]

In [35]:
pygame.quit()

In [ ]:
#video.ipython_display(fps=20, autoplay=1,maxduration=30)

## Saving video

In [ ]:
video.write_videofile("test2.mp4", temp_audiofile="temp-audio.m4a", remove_temp=True, codec="libx264", audio_codec="aac")